In [67]:
from IPython.core.display import display, HTML
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))


from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [68]:
from IPython.core.display import display, HTML
#display(HTML(my_html))     # make sure Jupyter knows to display it as HTML

In [69]:
listings_name = []
listings_price = []

In [70]:
listings_address_district=[]
listings_remaining_info = []

listings_type = []
listings_ri2 = []

listings_age = []
listings_ri3 = []

listings_bednbath = []
listings_ri4 = []

listings_size = []


In [71]:
def NamePriceSeparator(listings_namenprice):
    for i in range(len(listings_namenprice)):
        if i%2 ==0:
            listings_name.append(listings_namenprice[i].text)
        else:
            listings_price.append(listings_namenprice[i].text)
    return
    

In [72]:
totalpages = 70
for i in range(1, totalpages + 1):
    i = str(i)
    url = 'https://www.99.co/singapore/sale/condos-apartments?page_num=' + i 
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    listings_namenprice = soup.find_all('h3', class_="Z0npN _1T6Ql")
    NamePriceSeparator (listings_namenprice)
    
    
    listings = soup.find_all('p', class_="JPolj TMSN-")   

    for i in range(len(listings)):
        if i%6 ==0:
            listings_address_district.append(listings[i].text)
        else:
            listings_remaining_info.append(listings[i].text)
            
    for i in range(len(listings_remaining_info)):
        if i%5 == 0:
            listings_type.append(listings_remaining_info[i])
        else:
            listings_ri2.append(listings_remaining_info[i])
            
    for i in range(len(listings_ri2)):
        if i%4 == 0:
            listings_age.append(listings_ri2[i])
        else:
            listings_ri3.append(listings_ri2[i])
            
    for i in range(len(listings_ri3)):
        if i%3 == 0:
            listings_bednbath.append(listings_ri3[i])
        else:
            listings_ri4.append(listings_ri3[i])

    for i in range(len(listings_ri4)):
        if i%2 != 0:
            listings_size.append(listings_ri4[i])
   
    listings_remaining_info = []
    listings_ri2 = []
    listings_ri3 = []
    listings_ri4 = []

In [74]:
#Take out the '$' and ',' and convert to int.
for item in range(len(listings_price)):
    listings_price[item] = int(listings_price[item].replace('$', '').replace(',', ''))

In [75]:
#split the listings_address_district into address and District and put them in separate lists
listings_Address = []
listings_District = []
listings_address_district =([select for size in listings_address_district for select in size.split('·')])
for item in range(len(listings_address_district)):
    if (item % 2 == 0):
        listings_Address.append(listings_address_district[item])
    else:
        listings_District.append(listings_address_district[item])



In [76]:
#some bugger went to key in Floor #** instead of D*, so we replace them with a certain District D0
x = 'Floor'
for n, item in enumerate(listings_District):
    if x in item:
        listings_District[n] = ' D0'
        listings_District[n] = str(listings_District[n])

In [77]:
#split the listings_age into Completion Date(YYYY) and type of Leasehold and put them in separate lists
listings_CompletionYr = []
listings_Leasehold = []
listings_age =([select for size in listings_age for select in size.split('·')])
listings_age
for item in range(len(listings_age)):
    if (item % 2 == 0):
        listings_CompletionYr.append(listings_age[item])
    else:
        listings_Leasehold.append(listings_age[item])

In [78]:
for item in range(len(listings_bednbath)):
    if listings_bednbath[item] == '':
        listings_bednbath[item] = ('Na 0 ')
        


In [79]:
#split the listings_bednbath into respective categories and put them in separate lists
listings_bedroom =[]
listings_bathroom =[]

listings_bednbath = [select for room in listings_bednbath for select in room.split(' ')]
#After splitting, there is an empty '' in the order of consecutive 3rd element in the list, hence we drop the ''
listings_bednbath[:] = [item for item in listings_bednbath if item != '']

for item in range(len(listings_bednbath)):
    if (item % 2 == 0):
        listings_bedroom.append(listings_bednbath[item])
    else:
        listings_bathroom.append(listings_bednbath[item])

In [80]:
#convert listings_bathroom into int
for item in range(len(listings_bathroom)):
    listings_bathroom[item] = int(listings_bathroom[item])

In [81]:
#split the listings_size into sqft and sqm and put them in separate lists
listings_sqft = []
listings_sqm = []
listings_size =([select for size in listings_size for select in size.split('/')])

for item in range(len(listings_size)):
    if (item % 2 == 0):
        listings_sqft.append(listings_size[item])
    else:
        listings_sqm.append(listings_size[item])
        
        

In [82]:
#Remove ',' and ' sqft ' and convert to int
for item in range(len(listings_sqft)):
    listings_sqft[item] =int(listings_sqft[item].replace(',','').replace(' sqft ','')) 

In [83]:
#Remove ' ' and ' sqm ' and convert to float
for item in range(len(listings_sqm)):
    listings_sqm[item] =float(listings_sqm[item].replace(',','').replace(' ','').replace('sqm','')) 

In [84]:
#Now we combine all the lists into a dictionary
data = []
data = [{'Name': nam, 'Price': pri, 'Address': add, 'District': dis, 'Completion_Year':com, 'Leasehold':lea, 'Bedroom':bed, 'Bathroom':bath, 
         'Sqft':sqf, 'Sqm':sqm } for nam, pri, add, dis, com, lea, bed, bath, sqf, sqm in zip(listings_name, listings_price, listings_Address, 
         listings_District, listings_CompletionYr, listings_Leasehold, listings_bedroom, listings_bathroom, listings_sqft, listings_sqm)
       ]

In [86]:
df = pd.DataFrame(data)

In [87]:
df.shape

(2450, 10)

In [88]:
df.to_csv('outputversion.csv')